Need to do: 
1). Create Player History data (Last 3 years)
2). Create Get Live Current Season Data 
3). Create get methods for players 
4). Create sort methods by stats

In [12]:
pip install pandas requests 

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
###SCRAPING THE HISTORICAL DATA

import requests
import pandas as pd
from io import StringIO

# Function to download and process player data for specific seasons
def download_and_process_player_data(player_name):
    history_url = f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/players/{player_name}/history.csv"
    
    # Try to download the history.csv file
    history_response = requests.get(history_url)
    
    if history_response.status_code == 200:
        # Load the CSV data into a DataFrame
        player_data = pd.read_csv(StringIO(history_response.text))
        
        # Filter data by seasons and save to different files
        for season in ['2021/22', '2022/23', '2023/24']:
            season_data = player_data[player_data['season_name'] == season]
            
            if not season_data.empty:
                # Add a new column for the player's name
                season_data.insert(0, 'Name', player_name)
                
                # Determine the file name based on the season
                file_name = f'player_data_{season.replace("/", "_")}.csv'
                
                # Append to the file for this season
                season_data.to_csv(file_name, mode='a', index=False, header=not pd.io.common.file_exists(file_name))
                print(f"Appended data for {player_name} to {file_name}.")
    else:
        print(f"Could not find history.csv for {player_name}")

# Function to get all player folders from the 2023-24 directory
def get_player_folders():
    url = "https://api.github.com/repos/vaastav/Fantasy-Premier-League/contents/data/2023-24/players"
    
    # Send a GET request to the GitHub API
    response = requests.get(url)

    if response.status_code == 200:
        player_folders = response.json()  # Get the list of player folders
        return [player['name'] for player in player_folders if player['type'] == 'dir']  # Return only folder names
    else:
        print("Failed to retrieve player folders.")
        return []

# Get all player folders for the 2023-24 season
player_folders = get_player_folders()

# Loop through each player folder and process data
for player in player_folders:
    download_and_process_player_data(player)

In [ ]:
import os
import requests

def download_gw_csv_files(base_url, seasons, gw_range):
    """
    Downloads gameweek CSV files from the raw GitHub URL for specified seasons and gameweeks,
    and saves them in a directory in the current working directory.

    Parameters:
    base_url (str): The base URL for the raw GitHub content (e.g., 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/').
    seasons (list): A list of seasons (e.g., ['2021-22', '2022-23', '2023-24']).
    gw_range (range): The range of gameweeks to download (e.g., range(1, 39)).
    """
    # Get current working directory (CWD)
    cwd = os.getcwd()
    # Create a master folder named gw_data inside the CWD
    master_folder = os.path.join(cwd, 'gw_data')

    # Create master folder if it doesn't exist
    if not os.path.exists(master_folder):
        os.makedirs(master_folder)

    for season in seasons:
        # Create season folder inside master folder
        season_folder = os.path.join(master_folder, season)
        if not os.path.exists(season_folder):
            os.makedirs(season_folder)

        for gw in gw_range:
            gw_file = f'gw{gw}.csv'
            gw_url = f'{base_url}{season}/gws/{gw_file}'

            # Download the CSV file from the raw GitHub URL
            response = requests.get(gw_url)

            # Check if the request was successful
            if response.status_code == 200:
                # Save the file to the appropriate folder
                with open(os.path.join(season_folder, gw_file), 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {gw_file} for {season}")
            else:
                print(f"Failed to download {gw_file} for {season}: {response.status_code} (URL: {gw_url})")

# Example usage
base_url = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/'
seasons = ['2021-22', '2022-23', '2023-24']
gw_range = range(1, 39)  # Gameweeks 1 to 38

download_gw_csv_files(base_url, seasons, gw_range)
